<a href="https://colab.research.google.com/github/nicolaiberk/nlpdl_project/blob/main/01_Training_PPRs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install torch
!pip install sklearn

# code is based on Hauke Licht's CAP Model (https://colab.research.google.com/drive/1n7yHr0-lq-hmsXe2sqxLUhq7_4ejLH9o?usp=sharing#scrollTo=CiaoE0V8XZc6)

In [ ]:
# define model
from transformers import BertForSequenceClassification
checkpoint_model = 'distilbert-base-german-cased'
model = BertForSequenceClassification.from_pretrained(checkpoint_model)
model.train()

In [28]:
# load and prepare data
import pandas as pd

df = pd.read_csv('drive/MyDrive/germanyPPRs.csv', engine="python")
df = df.reset_index()
df = df.dropna()
# df = df.sample(1000) # drop this for full model
df.date = pd.to_datetime([dt for dt in df.date], format='%Y-%m-%d')

In [29]:
# restrict to press releases since 2013 to match newspaper data
mindate = pd.to_datetime('2013-01-01', format='%Y-%m-%d')
df = df[df.date >= mindate]

In [30]:
# train-test split by date
splitdate = pd.to_datetime('2018-01-01', format='%Y-%m-%d')
train = df[df.date < splitdate]
test = df[df.date >= splitdate]


In [31]:
train_texts = list(train["text"])
train_labels = list(train["label"])

test_texts = list(test["text"])
test_labels = list(test["label"])


# there are probably better ways to do this
ulabels = list(set(train_labels))
train_label_dict = {}

for i in range(len(ulabels)):
  train_label_dict[str(ulabels[i])] = i

train_labels = [train_label_dict[str(l)] for l in train_labels]


# and for test set
ulabels = list(set(test_labels))
test_label_dict = {}

for i in range(len(ulabels)):
  test_label_dict[str(ulabels[i])] = i

test_labels = [test_label_dict[str(l)] for l in test_labels]

In [32]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.2, random_state=1234, stratify=train_labels)

print(f"No. obs    -- test: {len(test_labels)}; train: {len(train_labels)}; validation: {len(val_labels)};")
print(f"No. labels -- test: {len(set(test_labels))}; train: {len(set(train_labels))}; validation: {len(set(val_labels))};")

No. obs    -- test: 7052; train: 16522; validation: 4131;
No. labels -- test: 6; train: 6; validation: 6;


In [33]:
# load the tokenizer 
from transformers import DistilBertTokenizerFast

tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-german-cased")

# tokenize texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)

In [34]:
import torch
class PPRDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = PPRDataset(train_encodings, train_labels)
val_dataset = PPRDataset(val_encodings, val_labels)
test_dataset = PPRDataset(test_encodings, test_labels)

In [35]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, n = precision_recall_fscore_support(labels, preds, average=None)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall,
        'n': n
    }

In [36]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
import os

training_args = TrainingArguments(
    # output directory
    output_dir=os.path.join("trained", "distilbert_cased", "results"),
    # total number of training epochs
    num_train_epochs=3,
    # batch size per device during training
    per_device_train_batch_size=16,
    # batch size for evaluation
    per_device_eval_batch_size=64,
    # number of warmup steps for learning rate scheduler
    warmup_steps=500,
    # strength of weight decay
    weight_decay=0.01,     
    # directory for storing logs
    logging_dir=os.path.join("trained", "distilbert_cased", "logs"),            
    logging_steps=250,
)


In [37]:
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-german-cased", num_labels = len(set(test_labels)))

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

Some weights of the model checkpoint at distilbert-base-german-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-german-cased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias

In [38]:
trainer.train()

Step,Training Loss
250,1.219700
500,0.143500
750,0.051000
1000,0.045300
1250,0.022800
1500,0.019300
1750,0.025700
2000,0.024400
2250,0.005900
2500,0.006300


TrainOutput(global_step=3099, training_loss=0.12846818071059465, metrics={'train_runtime': 2612.1121, 'train_samples_per_second': 1.186, 'total_flos': 1.0263315977367552e+16, 'epoch': 3.0, 'init_mem_cpu_alloc_delta': 4096, 'init_mem_gpu_alloc_delta': 270538240, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -1900814336, 'train_mem_gpu_alloc_delta': 806192640, 'train_mem_cpu_peaked_delta': 1906405376, 'train_mem_gpu_peaked_delta': 6683248128})

## Validation performance

In [39]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
eval_res = trainer.evaluate()

Trainer is attempting to log a value of "[0.99594046 1.         0.99341624 0.9984472  0.99750125 0.99331104]" of type <class 'numpy.ndarray'> for key "eval/f1" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.99594046 1.         0.99414348 1.         0.998      0.98671096]" of type <class 'numpy.ndarray'> for key "eval/precision" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[0.99594046 1.         0.99269006 0.99689922 0.997003   1.        ]" of type <class 'numpy.ndarray'> for key "eval/recall" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "[ 739  765  684  645 1001  297]" of type <class 'numpy.ndarray'> for key "eval/n" as a scalar. This invocation of Tensorboard's writer.add_sc

In [40]:
evaluated = pd.DataFrame()
evaluated["label"] = [v for k,v in train_label_dict.items()]
evaluated["class"] = [k for k,v in train_label_dict.items()]
evaluated["f1"] = eval_res["eval_f1"]
evaluated["precision"] = eval_res["eval_precision"]
evaluated["recall"] = eval_res["eval_recall"]
evaluated["n"] = eval_res["eval_n"]

In [41]:
evaluated.sort_values(by="f1", ascending=False)

,label,class,f1,precision,recall,n
1,1,SPD,1.000000,1.000000,1.000000,765
3,3,FDP,0.998447,1.000000,0.996899,645
4,4,Linke,0.997501,0.998000,0.997003,1001
0,0,Greens,0.995940,0.995940,0.995940,739
2,2,Union,0.993416,0.994143,0.992690,684
5,5,AfD,0.993311,0.986711,1.000000,297


In [42]:
trainer.save_model(os.path.join("drive", "MyDrive", "nlpdl", "01_PPR_model.bin"))

## Test set performance

In [43]:
eval_res = trainer.predict(test_dataset)

In [45]:
eval_res.metrics

{'test_accuracy': 0.9906409529211572,
 'test_f1': array([0.99513973, 0.98944193, 0.97905028, 0.99678457, 0.99149453,
        0.98971112]),
 'test_loss': 0.04879547655582428,
 'test_mem_cpu_alloc_delta': 425984,
 'test_mem_cpu_peaked_delta': 0,
 'test_mem_gpu_alloc_delta': 0,
 'test_mem_gpu_peaked_delta': 2618045952,
 'test_n': array([ 820,  661,  706, 1090, 1228, 2547]),
 'test_precision': array([0.99152542, 0.98646617, 0.96556474, 0.99816007, 0.98630137,
        0.9976067 ]),
 'test_recall': array([0.99878049, 0.9924357 , 0.99291785, 0.99541284, 0.99674267,
        0.98193954]),
 'test_runtime': 143.2269,
 'test_samples_per_second': 49.237}

In [46]:
evaluated = pd.DataFrame()
evaluated["label"] = [v for k,v in test_label_dict.items()]
evaluated["class"] = [k for k,v in test_label_dict.items()]
evaluated["f1"] = eval_res.metrics["test_f1"]
evaluated["precision"] = eval_res.metrics["test_precision"]
evaluated["recall"] = eval_res.metrics["test_recall"]
evaluated["n"] = eval_res.metrics["test_n"]

evaluated.sort_values(by="f1", ascending=False)

,label,class,f1,precision,recall,n
3,3,FDP,0.996785,0.998160,0.995413,1090
0,0,Greens,0.995140,0.991525,0.998780,820
4,4,Linke,0.991495,0.986301,0.996743,1228
5,5,AfD,0.989711,0.997607,0.981940,2547
1,1,SPD,0.989442,0.986466,0.992436,661
2,2,Union,0.979050,0.965565,0.992918,706


In [47]:
pd.crosstab(eval_res.label_ids, eval_res.predictions.argmax(-1))

col_0,0,1,2,3,4,5
row_0,,,,,,
0,819,0,1,0,0,0
1,0,656,2,0,2,1
2,2,1,701,0,0,2
3,2,0,3,1085,0,0
4,0,0,1,0,1224,3
5,3,8,18,2,15,2501
